<a href="https://colab.research.google.com/github/moni6264/deeplearning2/blob/main/CNN_PartA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: ignored

In [ ]:
#!find . -name "*.DS_Store" -type f -delete

KeyboardInterrupt: ignored

In [ ]:
zip_path = "drive/MyDrive/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip

In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization


!pip install wandb
import wandb
from keras.callbacks import EarlyStopping, ModelCheckpoint
from wandb.keras import WandbCallback
from keras.utils.vis_utils import plot_model
from PIL import Image

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
data_dir = "inaturalist_12K"
#data_augmentation = True
def data_preparation(data_dir , data_augmentation = True , batch_size = 250):
  train_dir = os.path.join(data_dir, "train")
  test_dir = os.path.join(data_dir, "val")
  if data_augmentation == True:
    train_datagen = ImageDataGenerator(rescale=1./255,
                                      height_shift_range=0.2,
                                      width_shift_range=0.2,
                                      horizontal_flip=True,
                                      zoom_range=0.2,
                                      fill_mode="nearest",
                                      validation_split = 0.1)
  else:
      train_datagen = ImageDataGenerator(rescale=1./255 ,validation_split = 0.1)
  test_datagen = ImageDataGenerator(rescale=1./255)


  train_generator = train_datagen.flow_from_directory(
          train_dir,
          target_size=(224, 224),
          shuffle=True,
          color_mode="rgb",
          batch_size=batch_size,
          class_mode='categorical',
          subset = "training")

  val_generator = train_datagen.flow_from_directory(
          train_dir,
          target_size=(224, 224),
          shuffle=True,
          color_mode="rgb",
          batch_size=batch_size,
          class_mode='categorical',
          subset = "validation")

  
  test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=batch_size)
  return train_generator , val_generator, test_generator

In [ ]:
def CNN(filter,filter_size,dense_layer,dropout, normalisation = True):


  model = Sequential()

   
  model.add(Conv2D(filter[0],filter_size[0], activation='relu', input_shape=(224, 224,3)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  if normalisation == True:
    model.add(BatchNormalization())
  model.add(Dropout(dropout))
  
  model.add(Conv2D(filter[1],filter_size[1], activation='relu'))
  if normalisation == True:
    model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(dropout))
  
  model.add(Conv2D(filter[2],filter_size[2], activation='relu'))
  if normalisation == True:
    model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Dropout(dropout))
  
  model.add(Conv2D(filter[3],filter_size[3], activation='relu'))
  if normalisation == True:
    model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(dropout))
 
  model.add(Conv2D(filter[4],filter_size[4], activation='relu'))
  if normalisation == True:
    model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(dropout))
  
  model.add(Flatten())
  model.add(Dense(dense_layer, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  

  #model.summary()
  return model

In [ ]:
#configuring wandb
%pip install wandb -q
import wandb
wandb.login()

wandb: Currently logged in as: fdl-moni_dip (use `wandb login --relogin` to force relogin)


True

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from wandb.keras import WandbCallback
def train():
    # Default values for hyper-parameters
    config_defaults = {
        "data_augmentation": True,
        "batch_size": 250,
        "normalisation": True,
        "dropout": 0.1,
        "filter": [16, 32, 64, 128, 256],
        "dense_layer": 256,
        "filters_size": [(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)]
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    # Local variables, values obtained from wandb config
    filter = config.filter
    data_augmentation = config.data_augmentation
    batch_size = config.batch_size
    normalisation = config.normalisation
    dropout = config.dropout
    dense_layer = config.dense_layer
    filters_size = config.filters_size
    
    # Display the hyperparameters
    run_name = "aug_{}_bs_{}_bn_{}_drop_{}_fc_{}_fil_{}_shape_{}".format(data_augmentation, batch_size, normalisation, dropout, dense_layer, filter, filters_size)
    print(run_name)

    # Create the data generators
    train_generator , val_generator, test_generator = data_preparation(data_dir , data_augmentation = True , batch_size = 250)
    model = CNN(filter,filters_size,dense_layer,dropout, normalisation)


    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Early Stopping callback
    earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')

    # To save the model with best validation accuracy
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
#   TRAIN_STEP_SIZE = train_generator.n//train_generator.batch_size
 #   VALIDATION_STEP_SIZE = validation_generator.n//validation_generator.batch_size
    history = model.fit(train_generator,
                        steps_per_epoch = train_generator.n//train_generator.batch_size,
                        validation_data = val_generator,
                        validation_steps = val_generator.n//val_generator.batch_size,
                        epochs=15, verbose = 2, 
                        callbacks=[WandbCallback(), earlyStopping, checkpoint])
                        #callbacks=[WandbCallback(input_type="image", generator=val_generator), earlyStopping, checkpoint])
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    
    # Meaningful name for the run
    wandb.run.name = run_name
    wandb.run.save()
    wandb.run.finish()
    return history

In [ ]:


# Sweep configuration
sweep_config = {
  "name": "CS6910 Assignment 2 - Part A",
  "metric": {
      "name":"val_accuracy",
      "goal": "maximize"
  },
  "method": "bayes",
  "parameters": {
        "data_augmentation": {
            "values": [True, False]
        },
        "batch_size": {
            "values": [128, 256]
        },
        "normalisation": {
            "values": [True, False]
        },
        "dropout": {
            "values": [0, 0.2, 0.1]
        },
        "filter": {
            "values": [[16, 32, 64, 128, 256], [32, 64, 128, 256, 512], [32, 32, 32, 32, 32],
                       [256, 128, 64, 32, 16], [64, 128, 256, 512, 1024], [128, 64, 32, 16, 8]]
        },
        "dense_layer": {
            "values": [128, 256, 512]
        },
        "filters_size": {
            "values": [[(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)],
                       [(7, 7), (5, 5), (3, 3), (3, 3), (3, 3)],
                       [(3, 3), (3, 3), (3, 3), (5, 5), (7, 7)]]
        }
    }
}

# Generates a sweep id
sweep_id = wandb.sweep(sweep_config, entity="dipnarayan501", project="dl_assignment_2")
wandb.agent(sweep_id, train, count=5)

Create sweep with ID: 2ura7hue
Sweep URL: https://wandb.ai/dipnarayan501/dl_assignment_2/sweeps/2ura7hue


wandb: Agent Starting Run: ifhrlmd7 with config:
wandb: 	batch_size: 256
wandb: 	data_augmentation: True
wandb: 	dense_layer: 256
wandb: 	dropout: 0
wandb: 	filter: [64, 128, 256, 512, 1024]
wandb: 	filters_size: [[7, 7], [5, 5], [3, 3], [3, 3], [3, 3]]
wandb: 	normalisation: False


aug_True_bs_256_bn_False_drop_0_fc_256_fil_[64, 128, 256, 512, 1024]_shape_[[7, 7], [5, 5], [3, 3], [3, 3], [3, 3]]
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/15



Run ifhrlmd7 errored: ResourceExhaustedError()
wandb: ERROR Run ifhrlmd7 errored: ResourceExhaustedError()
wandb: Agent Starting Run: lq3u4z33 with config:
wandb: 	batch_size: 128
wandb: 	data_augmentation: True
wandb: 	dense_layer: 512
wandb: 	dropout: 0
wandb: 	filter: [128, 64, 32, 16, 8]
wandb: 	filters_size: [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
wandb: 	normalisation: True


aug_True_bs_128_bn_True_drop_0_fc_512_fil_[128, 64, 32, 16, 8]_shape_[[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/15



Run lq3u4z33 errored: ResourceExhaustedError()
wandb: ERROR Run lq3u4z33 errored: ResourceExhaustedError()
wandb: Agent Starting Run: cwmmcknh with config:
wandb: 	batch_size: 128
wandb: 	data_augmentation: False
wandb: 	dense_layer: 128
wandb: 	dropout: 0.2
wandb: 	filter: [32, 64, 128, 256, 512]
wandb: 	filters_size: [[3, 3], [3, 3], [3, 3], [5, 5], [7, 7]]
wandb: 	normalisation: True


aug_False_bs_128_bn_True_drop_0.2_fc_128_fil_[32, 64, 128, 256, 512]_shape_[[3, 3], [3, 3], [3, 3], [5, 5], [7, 7]]
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/15
36/36 - 254s - loss: 2.7233 - accuracy: 0.1813 - val_loss: 2.4630 - val_accuracy: 0.1027 - _timestamp: 1648036361.0000 - _runtime: 264.0000 - 254s/epoch - 7s/step
Epoch 2/15
36/36 - 237s - loss: 2.1930 - accuracy: 0.2322 - val_loss: 2.3254 - val_accuracy: 0.1000 - _timestamp: 1648036598.0000 - _runtime: 501.0000 - 237s/epoch - 7s/step
Epoch 3/15


wandb: Ctrl + C detected. Stopping sweep.


In [ ]:

#wandb.init(project="test", entity="dipnarayan501", name="sample")

data_dir = "inaturalist_12K"
data_augmentation = True
# Number of filters in each layer
filter = [32, 64, 128, 256, 512]
# Number of neurons in the dense/fully connected layer
dense_layer = 512
# Shape of the filters used in each of the five layers
filter_size = [(3, 3), (3, 3), (3, 3), (5, 5), (7, 7)]

dropout = 0.1
normalisation = True
batch_size = 250

train_generator , val_generator, test_generator = data_preparation(data_dir , data_augmentation = True , batch_size = 250)
model = CNN(filter,filter_size,dense_layer,dropout, normalisation)


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Early Stopping callback
earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')

    # To save the model with best validation accuracy
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
#   TRAIN_STEP_SIZE = train_generator.n//train_generator.batch_size
 #   VALIDATION_STEP_SIZE = validation_generator.n//validation_generator.batch_size
history = model.fit(train_generator,
                        steps_per_epoch = train_generator.n//train_generator.batch_size,
                        validation_data = val_generator,
                        validation_steps = val_generator.n//val_generator.batch_size,
                        epochs=5, verbose = 2) 
                        #callbacks=[WandbCallback(data_type="image", generator=val_generator), earlyStopping, mc])





# Training the model

# Save the model
#model.save("best_model_dl_2")